# مروری کلی بر کرونا
> مروری بر کل موارد کرونا ، مرگ و میر و موارد جدید براساس کشور.

- comments: true
- author: پراتاب واردهان و سولماز عباسپور
- categories: [مرور]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

In [1]:
#hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')


Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/



In [2]:
#hide
!pip install jdatetime
!pip install persiantools
import numpy as np
import pandas as pd
from jinja2 import Template
import jdatetime as jd
import os
from persiantools import digits
from IPython.display import HTML

You should consider upgrading via the '/opt/hostedtoolcache/Python/3.6.12/x64/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/opt/hostedtoolcache/Python/3.6.12/x64/bin/python -m pip install --upgrade pip' command.


In [3]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
dirpath = os.path.dirname(os.getcwd())
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': dirpath + '/_templates/overview.tpl'
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [4]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [5]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

,Country/Region,Cases,Deaths,PCases,PDeaths,Cases (+),Deaths (+),Fatality Rate,Continent
0,US,24631888,410349,23758855,395851,873033,14498,1.7,North America
1,India,10625428,153032,10557985,152274,67443,758,1.4,Asia
2,Brazil,8697368,214147,8455059,209296,242309,4851,2.5,South America
3,Russia,3616680,66810,3507201,64134,109479,2676,1.8,Europe
4,United Kingdom,3553773,94765,3367070,88747,186703,6018,2.7,Europe
5,France,3046371,72139,2931686,70093,114685,2046,2.4,Europe
6,Spain,2456675,55041,2252164,53314,204511,1727,2.2,Europe
7,Italy,2428221,84202,2368733,81800,59488,2402,3.5,Europe
8,Turkey,2412505,24640,2380665,23832,31840,808,1.0,Asia
9,Germany,2108895,50876,2038645,46464,70250,4412,2.4,Europe


In [6]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [7]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
s_iran = df_table[df_table['Country/Region'].eq('Iran')][metrics].sum().add_prefix('Iran ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu, **s_iran}
summary

{'updated': Timestamp('2021-01-21 00:00:00'),
 'since': Timestamp('2021-01-16 00:00:00'),
 'Cases': 97461853,
 'Deaths': 2090076,
 'Cases (+)': 3021585,
 'Deaths (+)': 69350,
 'China Cases': 98786,
 'China Deaths': 4803,
 'China Cases (+)': 1011,
 'China Deaths (+)': 6,
 'US Cases': 24631888,
 'US Deaths': 410349,
 'US Cases (+)': 873033,
 'US Deaths (+)': 14498,
 'EU Cases': 28398034,
 'EU Deaths': 650487,
 'EU Cases (+)': 1059783,
 'EU Deaths (+)': 28034,
 'Iran Cases': 1354520,
 'Iran Deaths': 57150,
 'Iran Cases (+)': 30125,
 'Iran Deaths (+)': 433}

In [8]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,...,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21
Country/Region,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0,0,0,0,0,0,0,0,0,0,...,46,0,191,56,107,46,78,79,137,125
Albania,0,0,0,0,0,0,0,0,0,0,...,656,707,660,641,581,474,292,586,670,678
Algeria,0,0,0,0,0,0,0,0,0,0,...,272,219,267,254,230,222,259,249,265,246
Andorra,0,0,0,0,0,0,0,0,0,0,...,96,136,50,78,92,45,0,111,114,71
Angola,0,0,0,0,0,0,0,0,0,0,...,89,82,188,66,86,110,51,85,82,84


In [9]:
#hide_input
template = Template(get_template(paths['overview']))
days = digits.en_to_fa(str((summary['updated'] - summary['since']).days))
g_date_updated = pd.to_datetime(summary['updated'])
jdate_updated = jd.date.fromgregorian(year=g_date_updated.year,month=g_date_updated.month,day=g_date_updated.day,locale='fa_IR').strftime("%d / %m / %Y")
jdate_updated = digits.en_to_fa(jdate_updated)
g_date_since = pd.to_datetime(summary['since'])
jdate_since = jd.date.fromgregorian(year=g_date_since.year,month=g_date_since.month,day=g_date_since.day,locale='fa_IR').strftime("%d / %m / %Y")
jdate_since = digits.en_to_fa(jdate_since)
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, days=days, digits=digits, 
    jdate_updated=jdate_updated, jdate_since=jdate_since, enumerate=enumerate)
HTML(f'<div>{html}</div>')


تحلیل اصلی از [پراتاپ واردهان](https://twitter.com/PratapVardhan)[^۱]

[^۱]: منبع: ["2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). لینک به صفحه [اصلی](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-compare-country-trajectories.ipynb).